In [374]:
data = pd.read_csv('../data/test.csv')
data.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [375]:
data['Upc'] = data['Upc'].fillna(value = 0)
data['Upc'] = data['Upc'].astype(int)
data.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,72503389714,1,SHOES,3002.0
1,1,Friday,1707710732,1,DAIRY,1526.0
2,1,Friday,89470001026,1,DAIRY,1431.0
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2840015224,1,DSD GROCERY,4408.0


In [376]:
def check_len(upc):
    return len(str(upc))

In [377]:
data['len(upc)'] = data['Upc'].apply(check_len)

In [378]:
s = list(data['len(upc)'].unique())
s.sort()
for idx in s:

    count = len(data[data['len(upc)'].isin([idx])])
    print(idx," :  ",count)

1  :   3986
3  :   15
4  :   29564
5  :   286
7  :   1
8  :   400
9  :   2314
10  :   439012
11  :   169550
12  :   8518


In [379]:
data.shape

(653646, 7)

In [380]:
# check digit 구하는 함수

def get_checkdigit(upc):
    
    odd_s = [int(i) for i in str(upc)[-1::-2]]
    even_s = [int(i) for i in str(upc)[-2::-2]]
    
    odd_sum = sum(list(map(lambda number : number * 3, odd_s)))
    even_sum = sum(list(map(lambda number : number, even_s)))
    check_sum = (odd_sum + even_sum) % 10
    

    if check_sum == 0:
        return check_sum
    else:
        return 10 - check_sum

In [381]:
def make_full_upc(upc):
    if upc == 0:
        return "missing"
    else:
        tmp_upc = str(upc) + str(get_checkdigit(upc))


        if len(tmp_upc) < 12:
            length = 12 - len(tmp_upc)
            zero = "0" * length
            standard_upc = zero + tmp_upc
            return standard_upc
        else:
            return tmp_upc 

In [382]:
data['standard_upc'] = data['Upc'].apply(make_full_upc)
data['standard_upc(len)'] = data['standard_upc'].apply(check_len)

In [383]:
data['standard_upc(len)'].unique()

array([12, 13,  7])

In [384]:
def classify_company_product(standard_upc):
    class_ = []
    
    if standard_upc == "missing":
        return standard_upc
    
    elif len(standard_upc) == 13:
        company = standard_upc[2:7]
        product = standard_upc[7:12]
        
        class_.append(company)
        class_.append(product)
        return class_
    
    # len = 12
    else: 
        if standard_upc[:6] == "000000":
            company = standard_upc[-6:-3]
            product = standard_upc[-3:-1]
            class_.append(company)
            class_.append(product)
            return class_
        else:
            company = standard_upc[1:6]
            product = standard_upc[6:11]

            class_.append(company)
            class_.append(product)
            return class_

In [385]:
classify_company_product("9780989586566")

['80989', '58656']

In [386]:
classify_company_product("894700010267")

['94700', '01026']

In [387]:
data['tmp'] = data['standard_upc'].apply(classify_company_product)

In [388]:
data.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,len(upc),standard_upc,standard_upc(len),tmp
0,1,Friday,72503389714,1,SHOES,3002.0,11,725033897143,12,"[25033, 89714]"
1,1,Friday,1707710732,1,DAIRY,1526.0,10,017077107327,12,"[17077, 10732]"
2,1,Friday,89470001026,1,DAIRY,1431.0,11,894700010267,12,"[94700, 01026]"
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555.0,11,884912114709,12,"[84912, 11470]"
4,2,Friday,2840015224,1,DSD GROCERY,4408.0,10,028400152242,12,"[28400, 15224]"


In [389]:
def make_company_col(tmp):
    if tmp == "missing":
        return tmp
    else:
        return tmp[0]

def make_product_col(tmp):
    if tmp == "missing":
        return tmp
    else:
        return tmp[1]

data['company_code'] = data['tmp'].apply(make_company_col)
data['product_code'] = data['tmp'].apply(make_product_col)

In [390]:
data.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,len(upc),standard_upc,standard_upc(len),tmp,company_code,product_code
0,1,Friday,72503389714,1,SHOES,3002.0,11,725033897143,12,"[25033, 89714]",25033,89714
1,1,Friday,1707710732,1,DAIRY,1526.0,10,017077107327,12,"[17077, 10732]",17077,10732
2,1,Friday,89470001026,1,DAIRY,1431.0,11,894700010267,12,"[94700, 01026]",94700,01026
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555.0,11,884912114709,12,"[84912, 11470]",84912,11470
4,2,Friday,2840015224,1,DSD GROCERY,4408.0,10,028400152242,12,"[28400, 15224]",28400,15224


In [391]:
del data['Upc'], data['len(upc)'], data['tmp'], data['standard_upc(len)']

In [392]:
data.head()

,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,standard_upc,company_code,product_code
0,1,Friday,1,SHOES,3002.0,725033897143,25033,89714
1,1,Friday,1,DAIRY,1526.0,017077107327,17077,10732
2,1,Friday,1,DAIRY,1431.0,894700010267,94700,01026
3,1,Friday,1,GROCERY DRY GOODS,3555.0,884912114709,84912,11470
4,2,Friday,1,DSD GROCERY,4408.0,028400152242,28400,15224


In [558]:
data.to_csv('../data/test_v1.csv')

In [1]:
test = pd.read_csv('../data/test_v1.csv', index_col = 0)
test.head()

,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,standard_upc,company_code,product_code
0,1,Friday,1,SHOES,3002.0,725033897143,25033,89714
1,1,Friday,1,DAIRY,1526.0,017077107327,17077,10732
2,1,Friday,1,DAIRY,1431.0,894700010267,94700,01026
3,1,Friday,1,GROCERY DRY GOODS,3555.0,884912114709,84912,11470
4,2,Friday,1,DSD GROCERY,4408.0,028400152242,28400,15224


In [2]:
train = pd.read_csv('../data/train_v2.csv', index_col = 0)
train.head()

,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,standard_upc,company_code,product_code
0,999,5,Friday,-1,0,1000,681131529297,81131,52929
1,30,7,Friday,1,1,8931,605388159809,05388,15980
2,30,7,Friday,1,2,4504,074108110992,74108,11099
3,26,8,Friday,2,3,3565,022384035102,22384,03510
4,26,8,Friday,2,3,1017,020066137441,20066,13744


In [3]:
len(test), len(train)

(653646, 647052)

In [4]:
len(test['VisitNumber'].unique()), len(train['VisitNumber'].unique())

(95674, 95674)

In [5]:
def convert_description(description):
    
    description_list= ['FINANCIAL SERVICES', 'SHOES', 'PERSONAL CARE', 'PAINT AND ACCESSORIES', 'DSD GROCERY', 'MEAT - FRESH & FROZEN', 'DAIRY', 'PETS AND SUPPLIES', 'HOUSEHOLD CHEMICALS/SUPP', 'MISSING', 'IMPULSE MERCHANDISE', 'PRODUCE', 'CANDY, TOBACCO, COOKIES', 'GROCERY DRY GOODS', 'BOYS WEAR', 'FABRICS AND CRAFTS', 'JEWELRY AND SUNGLASSES', 'MENS WEAR', 'ACCESSORIES', 'HOME MANAGEMENT', 'FROZEN FOODS', 'SERVICE DELI', 'INFANT CONSUMABLE HARDLINES', 'PRE PACKED DELI', 'COOK AND DINE', 'PHARMACY OTC', 'LADIESWEAR', 'COMM BREAD', 'BAKERY', 'HOUSEHOLD PAPER GOODS', 'CELEBRATION', 'HARDWARE', 'BEAUTY', 'AUTOMOTIVE', 'BOOKS AND MAGAZINES', 'SEAFOOD', 'OFFICE SUPPLIES', 'LAWN AND GARDEN', 'SHEER HOSIERY', 'WIRELESS', 'BEDDING', 'BATH AND SHOWER', 'HORTICULTURE AND ACCESS', 'HOME DECOR', 'TOYS', 'INFANT APPAREL', 'LADIES SOCKS', 'PLUS AND MATERNITY', 'ELECTRONICS', 'GIRLS WEAR, 4-6X  AND 7-14', 'BRAS & SHAPEWEAR', 'LIQUOR,WINE,BEER', 'SLEEPWEAR/FOUNDATIONS', 'CAMERAS AND SUPPLIES', 'SPORTING GOODS', 'PLAYERS AND ELECTRONICS', 'PHARMACY RX', 'MENSWEAR', 'OPTICAL - FRAMES', 'SWIMWEAR/OUTERWEAR', 'OTHER DEPARTMENTS', 'MEDIA AND GAMING', 'FURNITURE', 'OPTICAL - LENSES', 'SEASONAL', 'LARGE HOUSEHOLD GOODS', '1-HR PHOTO', 'CONCEPT STORES', 'HEALTH AND BEAUTY AIDS']
    result = description_list.index(description)
    
    if result == 9:
        return 999
    else:
        return result

In [6]:
test.shape

(653646, 8)

In [7]:
test['DepartmentDescription'] = test['DepartmentDescription'].fillna(value = "MISSING")

In [8]:
test['DepartmentDescription'] = test['DepartmentDescription'].apply(convert_description)

In [9]:
def convert_missing(missing):
    if missing == "missing":
        return 999
    else:
        return missing

In [10]:
test['standard_upc'] = test['standard_upc'].apply(convert_missing)
test['company_code'] = test['company_code'].apply(convert_missing)
test['product_code'] = test['product_code'].apply(convert_missing)

test['FinelineNumber'] = test['FinelineNumber'].fillna(value = 999)
test['FinelineNumber'] = test['FinelineNumber'].astype(int)
test['FinelineNumber'] = test['FinelineNumber'].astype('category')
test['company_code'] = test['company_code'].fillna(value = 999)
test['company_code'] = test['company_code'].astype('category')
test['product_code'] = test['product_code'].fillna(value = 999)
test['product_code'] = test['product_code'].astype('category')

In [11]:
train['standard_upc'] = train['standard_upc'].apply(convert_missing)
train['company_code'] = train['company_code'].apply(convert_missing)
train['product_code'] = train['product_code'].apply(convert_missing)

train['FinelineNumber'] = train['FinelineNumber'].fillna(value = 999)
train['FinelineNumber'] = train['FinelineNumber'].astype(int)
train['FinelineNumber'] = train['FinelineNumber'].astype('category')
train['company_code'] = train['company_code'].fillna(value = 999)
train['company_code'] = train['company_code'].astype('category')
train['product_code'] = train['product_code'].fillna(value = 999)
train['product_code'] = train['product_code'].astype('category')

In [12]:
len(test['VisitNumber'].unique()), len(train['VisitNumber'].unique())

(95674, 95674)

In [13]:
#delete_index = train[train['company_code'].isin([9999])].index

In [14]:
#train = train.drop(index=delete_index)

In [15]:
len(test['VisitNumber'].unique()), len(train['VisitNumber'].unique())

(95674, 95674)

In [16]:
print(len(set(test['product_code'].unique()) - set(train['product_code'].unique())))
print(len(set(train['product_code'].unique()) - set(test['product_code'].unique())))

9021
8796


In [17]:
421,401

(421, 401)

In [18]:
print(len(set(test['company_code'].unique()) - set(train['company_code'].unique())))
print(len(set(train['company_code'].unique()) - set(test['company_code'].unique())))

421
401


In [19]:
print(len(set(test['FinelineNumber'].unique()) - set(train['FinelineNumber'].unique())))
print(len(set(train['FinelineNumber'].unique()) - set(test['FinelineNumber'].unique())))

158
150


In [20]:
print(len(set(test['DepartmentDescription'].unique()) - set(train['DepartmentDescription'].unique())))
print(len(set(train['DepartmentDescription'].unique()) - set(test['DepartmentDescription'].unique())))

0
0


In [21]:
print(len(set(test['VisitNumber'].unique()) - set(train['VisitNumber'].unique())))
print(len(set(train['VisitNumber'].unique()) - set(test['VisitNumber'].unique())))

95674
95674


In [22]:
a = list(train['TripType'].unique())
a.sort()
print(a)

[3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 999]


In [23]:
comcode_list1 = list(set(test['company_code'].unique()) - set(train['company_code'].unique()))
comcode_list2 = list(set(train['company_code'].unique()) - set(test['company_code'].unique()))

In [24]:
test['company_code'].unique()

[25033, 17077, 94700, 84912, 28400, ..., 91953, 35200, 12574, 87083, 64666]
Length: 5833
Categories (5833, object): [25033, 17077, 94700, 84912, ..., 35200, 12574, 87083, 64666]

In [25]:
delete_test = test[test['company_code'].isin(comcode_list1)].index
delete_train = train[train['company_code'].isin(comcode_list2)].index

In [26]:
test_df = test.drop(index = delete_test)
train_df = train.drop(index = delete_train)

In [27]:
global tmp
tmp = list(train_df['company_code'].unique())
def convert_comcode(comcode):
    global tmp
    try:
        return tmp.index(comcode)
    except:
        return 9999
train_df['company_code'] = train_df['company_code'].apply(convert_comcode)
test_df['company_code'] = test_df['company_code'].apply(convert_comcode)

In [28]:
company_set = train.groupby('TripType')["company_code"].apply(set)

In [29]:
tmp[0], tmp[1], tmp[63]

('81131', '05388', '34000')

In [30]:
company_set[3]&company_set[4]&company_set[5]&company_set[6]&company_set[7]&company_set[8]&company_set[9]&company_set[12]&company_set[14]&company_set[15]&company_set[18]&company_set[19]&company_set[20]&company_set[21]&company_set[22]&company_set[23]&company_set[24]&company_set[25]&company_set[26]&company_set[27]&company_set[28]&company_set[29]&company_set[30]&company_set[31]&company_set[32]&company_set[33]&company_set[34]&company_set[35]&company_set[36]&company_set[37]&company_set[38]&company_set[39]&company_set[40]&company_set[41]&company_set[42]&company_set[43]&company_set[44]&company_set[999]

{'05388', '34000', '81131'}

In [31]:
description_set = train.groupby('TripType')["DepartmentDescription"].apply(set)

In [32]:
description_set[3]&description_set[4]&description_set[5]&description_set[6]&description_set[7]&description_set[8]&description_set[9]&description_set[12]&description_set[14]&description_set[15]&description_set[18]&description_set[19]&description_set[20]&description_set[21]&description_set[22]&description_set[23]&description_set[24]&description_set[25]&description_set[26]&description_set[27]&description_set[28]&description_set[29]&description_set[30]&description_set[31]&description_set[32]&description_set[33]&description_set[34]&description_set[35]&description_set[36]&description_set[37]&description_set[38]&description_set[39]&description_set[40]&description_set[41]&description_set[42]&description_set[43]&description_set[44]&description_set[999]

{10, 29, 30, 36}

In [33]:
description_list = ['FINANCIAL SERVICES', 'SHOES', 'PERSONAL CARE', 'PAINT AND ACCESSORIES', 'DSD GROCERY', 'MEAT - FRESH & FROZEN', 'DAIRY', 'PETS AND SUPPLIES', 'HOUSEHOLD CHEMICALS/SUPP', 'MISSING', 'IMPULSE MERCHANDISE', 'PRODUCE', 'CANDY, TOBACCO, COOKIES', 'GROCERY DRY GOODS', 'BOYS WEAR', 'FABRICS AND CRAFTS', 'JEWELRY AND SUNGLASSES', 'MENS WEAR', 'ACCESSORIES', 'HOME MANAGEMENT', 'FROZEN FOODS', 'SERVICE DELI', 'INFANT CONSUMABLE HARDLINES', 'PRE PACKED DELI', 'COOK AND DINE', 'PHARMACY OTC', 'LADIESWEAR', 'COMM BREAD', 'BAKERY', 'HOUSEHOLD PAPER GOODS', 'CELEBRATION', 'HARDWARE', 'BEAUTY', 'AUTOMOTIVE', 'BOOKS AND MAGAZINES', 'SEAFOOD', 'OFFICE SUPPLIES', 'LAWN AND GARDEN', 'SHEER HOSIERY', 'WIRELESS', 'BEDDING', 'BATH AND SHOWER', 'HORTICULTURE AND ACCESS', 'HOME DECOR', 'TOYS', 'INFANT APPAREL', 'LADIES SOCKS', 'PLUS AND MATERNITY', 'ELECTRONICS', 'GIRLS WEAR, 4-6X  AND 7-14', 'BRAS & SHAPEWEAR', 'LIQUOR,WINE,BEER', 'SLEEPWEAR/FOUNDATIONS', 'CAMERAS AND SUPPLIES', 'SPORTING GOODS', 'PLAYERS AND ELECTRONICS', 'PHARMACY RX', 'MENSWEAR', 'OPTICAL - FRAMES', 'SWIMWEAR/OUTERWEAR', 'OTHER DEPARTMENTS', 'MEDIA AND GAMING', 'FURNITURE', 'OPTICAL - LENSES', 'SEASONAL', 'LARGE HOUSEHOLD GOODS', '1-HR PHOTO', 'CONCEPT STORES', 'HEALTH AND BEAUTY AIDS']

In [34]:
print(description_list[10])
print(description_list[29])
print(description_list[30])
print(description_list[36])

IMPULSE MERCHANDISE
HOUSEHOLD PAPER GOODS
CELEBRATION
OFFICE SUPPLIES


In [35]:
def product_info(description,comcode):
    
    if (description == 10 and comcode == 0) or (description == 10 and comcode == 1) or (description == 10 and comcode == 63):
        return "0_0"
    
    elif (description == 29 and comcode == 0) or (description == 29 and comcode == 1) or (description == 29 and comcode == 63):
        return "0_0"
    
    elif (description == 30 and comcode == 0) or (description == 30 and comcode == 1) or (description == 30 and comcode == 63):
        return "0_0"
    
    elif (description == 36 and comcode == 0) or (description == 36 and comcode == 1) or (description == 36 and comcode == 63):
        return "0_0"
    
    else:
        result = str(description) + "_" + str(comcode)
        return result

In [36]:
def product_info(description,comcode):
    
    result = str(description) + "_" + str(comcode)
    return result

In [37]:
test_df['product_info'] = test_df.apply(lambda row : product_info(row['DepartmentDescription'], row['company_code']), axis = 1)
train_df['product_info'] = train_df.apply(lambda row : product_info(row['DepartmentDescription'], row['company_code']), axis = 1)


In [38]:
len(set(train_df['product_info'].unique()) - set(test_df['product_info'].unique()))

138

In [39]:
def weekday(x):
    if x == "Saturday" or x == "Sumday":
        return 1
    else:
        return 0

In [40]:
test_df['Weekday'] = test_df['Weekday'].apply(weekday)
train_df['Weekday'] = train_df['Weekday'].apply(weekday)

In [41]:
set(train_df['company_code'].unique()) - set(test_df['company_code'].unique())

set()

In [42]:
set(test_df['company_code'].unique()) - set(train_df['company_code'].unique())

set()

In [43]:
len(train_df['company_code'].unique())

5412

In [44]:
global tmp
tmp = list(train_df['FinelineNumber'].unique())
def convert_fineline(fineline):
    global tmp
    try:
        return tmp.index(fineline)
    except:
        return 9999
train_df['FinelineNumber'] = train_df['FinelineNumber'].apply(convert_fineline)
test_df['FinelineNumber'] = test_df['FinelineNumber'].apply(convert_fineline)

In [45]:
train_df['refund'] = train_df['ScanCount'].apply(lambda x: -x if x < 0 else 0)
train_df['purchase'] = train_df['ScanCount'].apply(lambda x: x if x > 0 else 0)

test_df['refund'] = test_df['ScanCount'].apply(lambda x: -x if x < 0 else 0)
test_df['purchase'] = test_df['ScanCount'].apply(lambda x: x if x > 0 else 0)

In [46]:
del train_df['product_code'], train_df['product_info'], train_df['standard_upc'], train_df['ScanCount']
del test_df['product_code'], test_df['product_info'], test_df['standard_upc'] , test_df['ScanCount']

In [47]:
train_df.head()

,TripType,VisitNumber,Weekday,DepartmentDescription,FinelineNumber,company_code,refund,purchase
0,999,5,0,0,0,0,1,0
1,30,7,0,1,1,1,0,1
2,30,7,0,2,2,2,0,1
3,26,8,0,3,3,3,0,2
4,26,8,0,3,4,4,0,2


In [76]:
X = train_df.loc[:,"VisitNumber":"company_code"]
del X['Weekday']
X.head()

,VisitNumber,DepartmentDescription,FinelineNumber,company_code
0,5,0,0,0
1,7,1,1,1
2,7,2,2,2
3,8,3,3,3
4,8,3,4,4


In [75]:
X1 = X.loc[:200000,:]
X2 = X.loc[200001:400011,:]
X3 = X.loc[400012:,:]

In [77]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[False, True, True, True])

pre1 = ohe.fit_transform(X1)
pre2 = ohe.fit_transform(X2)
pre3 = ohe.fit_transform(X3)

df1 = pd.DataFrame(pre1.toarray())
df2 = pd.DataFrame(pre2.toarray())
df3 = pd.DataFrame(pre3.toarray())

In [ ]:
df1 = df1.rename(columns={9438 : "VisitNumber"})
df2 = df2.rename(columns={9438 : "VisitNumber"})
df3 = df3.rename(columns={9438 : "VisitNumber"})

df1 = df1.groupby(df1['VisitNumber']).sum()
df2 = df2.groupby(df2['VisitNumber']).sum()
df3 = df3.groupby(df3['VisitNumber']).sum()

In [67]:
df4 = train_df[['VisitNumber', 'TripType','Weekday']].groupby('VisitNumber').mean()
df5 = train_df[['VisitNumber', 'refund','purchase']].groupby('VisitNumber').sum()

In [111]:
df2 = train_df

,TripType,Weekday
VisitNumber,,
5,999,0
7,30,0
8,26,0
9,8,0
10,8,0
11,35,0
12,41,0
15,21,0
17,6,0


In [122]:
del test['standard_upc'], test['company_code'], test['product_code'], test['FinelineNumber'], test['DepartmentDescription']

In [123]:
del train['standard_upc'], train['company_code'], train['product_code'], train['FinelineNumber'], train['DepartmentDescription']

In [124]:
len(test['VisitNumber'].unique()), len(train['VisitNumber'].unique())

(95674, 95674)

In [125]:
test.head()

,VisitNumber,Weekday,ScanCount,product_info
0,1,0,1,1_34
1,1,0,1,6_3743
2,1,0,1,6_51
3,1,0,1,13_373
4,2,0,1,4_26


In [126]:
train.head()

,TripType,VisitNumber,Weekday,ScanCount,product_info
0,999,5,0,-1,0_0
1,30,7,0,1,1_0
2,30,7,0,1,2_2
3,26,8,0,2,3_3
4,26,8,0,2,3_4


In [52]:
test.shape

(653646, 4)

In [53]:
len(test['VisitNumber'].unique()), len(train['VisitNumber'].unique())

(95674, 95674)

In [54]:
#delete_info_list = list(set(train['product_info'].unique()) - set(test['product_info'].unique()))

In [55]:
#delete_info_list2 = list(set(test['product_info'].unique()) - set(train['product_info'].unique()))

In [56]:
#delete_index1 = train[train['product_info'].isin(delete_info_list)].index

In [57]:
#delete_index2 = test[test['product_info'].isin(delete_info_list2)].index

In [58]:
#train_df = train.drop(index = delete_index1)
#test_df = test.drop(index = delete_index2)

In [129]:
train_df = train.copy()
test_df = test.copy()

In [130]:
len(set(train_df['product_info'].unique()) - set(test_df['product_info'].unique()))

541

In [131]:
len(set(test_df['product_info'].unique()) - set(train_df['product_info'].unique()))

171

In [61]:
len(test_df['VisitNumber'].unique()), len(train_df['VisitNumber'].unique())

(95565, 95574)

In [127]:
global info_list
info_list = list(train_df['product_info'].unique())
def convert_product_info(info):

    return info_list.index(info)

train_df['product_info'] = train_df['product_info'].apply(convert_product_info)
test_df['product_info'] = test_df['product_info'].apply(convert_product_info)

train_df['refund'] = train_df['ScanCount'].apply(lambda x: -x if x < 0 else 0)
train_df['purchase'] = train_df['ScanCount'].apply(lambda x: x if x > 0 else 0)

test_df['refund'] = test_df['ScanCount'].apply(lambda x: -x if x < 0 else 0)
test_df['purchase'] = test_df['ScanCount'].apply(lambda x: x if x > 0 else 0)

del train_df['ScanCount']
del test_df['ScanCount']

NameError: name 'train_df' is not defined

In [63]:
train_df.head()

,TripType,VisitNumber,Weekday,product_info,refund,purchase
0,999,5,0,0,1,0
1,30,7,0,1,0,1
2,30,7,0,2,0,1
3,26,8,0,3,0,2
4,26,8,0,4,0,2


In [64]:
test_df.head()

,VisitNumber,Weekday,product_info,refund,purchase
0,1,0,34,0,1
1,1,0,4502,0,1
2,1,0,55,0,1
3,1,0,434,0,1
4,2,0,38,0,1


In [65]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[False, False, False, True, False, False])

pre_X = ohe.fit_transform(train_df)

train_df = pd.DataFrame(pre_X.toarray())
train_df.head()


,0,1,2,3,4,5,6,7,8,9,...,6821,6822,6823,6824,6825,6826,6827,6828,6829,6830
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,999.0,5.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,30.0,7.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,30.0,7.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,26.0,8.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,26.0,8.0,0.0,0.0,2.0


In [ ]:
train_df = train_df.rename(columns = {6826:"TripType", 6827 : "VisitNumber", 6828 : "Weekday", 6829 : "refund", 6830 : "purchase"})
train_df.head()

In [543]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[False, False, True, False, False])

pre_X = ohe.fit_transform(test_df)

test_df = pd.DataFrame(pre_X.toarray())
test_df.head()


,0,1,2,3,4,5,6,7,8,9,...,6820,6821,6822,6823,6824,6825,6826,6827,6828,6829
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0


In [ ]:
test_df = test_df.rename(columns = {6826 : "VisitNumber", 6827 : "Weekday", 6828 : "refund", 6829 : "purchase"})
test_df.head()

In [ ]:
(95674, 95674)

In [ ]:
train_df = train_df.rename(columns = {7367:"TripType", 7368 : "VisitNumber", 7369 : "Weekday", 7370 : "refund", 7371 : "purchase"})
train_df.head()

In [ ]:
del data['standard_upc']

In [ ]:
data[data['DepartmentDescription'].isnull()]

In [ ]:
data[data['FinelineNumber'].isnull()]